# Parte A

## Questão 4 - Relembre e registe, de forma sucinta, os principais problemas de qualidade identificados na Ficha nº 1.

### Síntese — principais problemas de qualidade identificados

- **Valores anómalos/extremos**: distâncias e montantes claramente fora do esperado.
- **Inconsistências temporais**: viagens com fim antes do início ou datas fora do período do dataset.
- **Valores em falta**: presença de valores nulos em variáveis relevantes para análise.
- **Registos potencialmente inválidos**: duração negativa/nula e total da viagem igual a zero ou negativo.

In [100]:
import pandas as pd
import os

# Load the dataset
df = pd.read_parquet("../Worksheet_1/yellow_tripdata_2025-01.parquet")
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,1,2025-01-01 00:18:38,2025-01-01 00:26:59,1.0,1.60,1.0,N,229,237,1,10.0,3.5,0.5,3.00,0.0,1.0,18.00,2.5,0.0,0.0
1,1,2025-01-01 00:32:40,2025-01-01 00:35:13,1.0,0.50,1.0,N,236,237,1,5.1,3.5,0.5,2.02,0.0,1.0,12.12,2.5,0.0,0.0
2,1,2025-01-01 00:44:04,2025-01-01 00:46:01,1.0,0.60,1.0,N,141,141,1,5.1,3.5,0.5,2.00,0.0,1.0,12.10,2.5,0.0,0.0
3,2,2025-01-01 00:14:27,2025-01-01 00:20:01,3.0,0.52,1.0,N,244,244,2,7.2,1.0,0.5,0.00,0.0,1.0,9.70,0.0,0.0,0.0
4,2,2025-01-01 00:21:34,2025-01-01 00:25:06,3.0,0.66,1.0,N,244,116,2,5.8,1.0,0.5,0.00,0.0,1.0,8.30,0.0,0.0,0.0


# Parte B

## Questão 5 - Meça o consumo de memória do DataFrame antes de qualquer limpeza.

In [101]:
memory_before_mb = df.memory_usage(deep=True).sum() / (1024 ** 2)
print(f"Memory usage before cleaning/optimization: {memory_before_mb:.2f} MB")

Memory usage before cleaning/optimization: 616.31 MB


## Questão 6 - Analise os tipos de dados atribuídos automaticamente às colunas numéricas e temporais e identifique, justificando, colunas cujo tipo possa ser otimizado.

In [102]:
df.dtypes

VendorID                          int32
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int32
DOLocationID                      int32
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
Airport_fee                     float64
cbd_congestion_fee              float64
dtype: object

Colunas numéricas como `passenger_count`, `trip_distance`, `fare_amount`, `tip_amount` e `total_amount` podem ser otimizadas para tipos de dados mais eficientes, como `int8` ou `float32`, dependendo do intervalo de valores presentes. Colunas temporais como `tpep_pickup_datetime` e `tpep_dropoff_datetime` devem ser convertidas para o tipo `datetime64[ns]` para otimizar o armazenamento e facilitar operações temporais.

## Questão 7 - Aplique otimizações de tipos de dados (por exemplo, conversão de float64 para float32 ou uso de tipos categóricos quando adequado).

In [103]:
float_columns = ['tip_amount', 'passenger_count', 'RatecodeID', 'extra', 'mta_tax', 'improvement_surcharge', 'congestion_surcharge', 'tolls_amount', 'Airport_fee', 'cbd_congestion_fee']
for col in float_columns:
    df[col] = df[col].astype('float32')

categorical_columns = ['PULocationID', 'DOLocationID']
for col in categorical_columns:
    df[col] = df[col].astype('category')

int_columns = ['VendorID', 'payment_type']
for col in int_columns:
    df[col] = df[col].astype('int8')

'''
Bloco para analisar os valores únicos de cada coluna para verificar se a conversão para `float32` é adequada ou se há valores que possam ser otimizados usando um tipo de dado diferente (por exemplo, `int8` ou `category`).

valores = df['coluna'].unique().tolist()

print(f"Unique values in 'coluna': {valores}")
'''

df.dtypes

VendorID                           int8
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float32
trip_distance                   float64
RatecodeID                      float32
store_and_fwd_flag               object
PULocationID                   category
DOLocationID                   category
payment_type                       int8
fare_amount                     float64
extra                           float32
mta_tax                         float32
tip_amount                      float32
tolls_amount                    float32
improvement_surcharge           float32
total_amount                    float64
congestion_surcharge            float32
Airport_fee                     float32
cbd_congestion_fee              float32
dtype: object

## Questão 8 - Converta todas as variáveis identificadas como temporais para um tipo de dado temporal adequado.

In [104]:
datetime_columns = ['tpep_pickup_datetime', 'tpep_dropoff_datetime']

datetime_blank_counts = {}
for col in datetime_columns:
    raw_values = df[col].astype('string')
    datetime_blank_counts[col] = raw_values.str.fullmatch(r'\s*').fillna(False).sum()
    df[col] = pd.to_datetime(raw_values, errors='coerce')

## Questão 9 - Verifique a existência de valores temporais nulos, inválidos ou incoerentes após a conversão.

In [105]:
for col in datetime_columns:
    print(f"Null values in '{col}': {df[col].isnull().sum()}")
    print(f"Invalid values in '{col}': {df[col].apply(lambda x: 1 if pd.isna(x) else 0).sum()}")
    print(f"Blank/whitespace values in '{col}' before conversion: {datetime_blank_counts.get(col, 0)}")

Null values in 'tpep_pickup_datetime': 0
Invalid values in 'tpep_pickup_datetime': 0
Blank/whitespace values in 'tpep_pickup_datetime' before conversion: 0
Null values in 'tpep_dropoff_datetime': 0
Invalid values in 'tpep_dropoff_datetime': 0
Blank/whitespace values in 'tpep_dropoff_datetime' before conversion: 0


## Questão 10 - Trate os registos problemáticos identificados, justificando a estratégia adotada (remoção ou correção).

In [106]:
# Estratégia adotada:
# - Remoção de registos com inconsistências temporais (datas inválidas ou fim < início)
# - Remoção de registos fora do período de janeiro de 2025
# - Remoção de registos com duração inválida (<= 0 min ou > 300 min)
# - Remoção de registos com distância/valor total inválidos (<= 0)

before_rows = len(df)
df_clean = df.copy()

# Reforço da conversão temporal (seguro, em caso de reexecução fora de ordem)
for col in datetime_columns:
    df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')

# Duração da viagem (minutos)
df_clean['trip_duration_min'] = (
    df_clean['tpep_dropoff_datetime'] - df_clean['tpep_pickup_datetime']
).dt.total_seconds() / 60

jan_start = pd.Timestamp('2025-01-01')
feb_start = pd.Timestamp('2025-02-01')

invalid_datetime_mask = (
    df_clean['tpep_pickup_datetime'].isna() |
    df_clean['tpep_dropoff_datetime'].isna() |
    (df_clean['tpep_dropoff_datetime'] < df_clean['tpep_pickup_datetime'])
)

invalid_period_mask = (
    (df_clean['tpep_pickup_datetime'] < jan_start) |
    (df_clean['tpep_pickup_datetime'] >= feb_start) |
    (df_clean['tpep_dropoff_datetime'] < jan_start) |
    (df_clean['tpep_dropoff_datetime'] >= feb_start)
)

invalid_duration_mask = (
    df_clean['trip_duration_min'].isna() |
    (df_clean['trip_duration_min'] <= 0) |
    (df_clean['trip_duration_min'] > 300)
)

invalid_business_mask = (
    (df_clean['trip_distance'] <= 0) |
    (df_clean['total_amount'] <= 0)
)

invalid_mask = (
    invalid_datetime_mask |
    invalid_period_mask |
    invalid_duration_mask |
    invalid_business_mask
)

removed_rows = int(invalid_mask.sum())
df_clean = df_clean.loc[~invalid_mask].copy()
after_rows = len(df_clean)

print("Limpeza concluída.")
print(f"Registos antes: {before_rows:,}")
print(f"Registos removidos: {removed_rows:,}")
print(f"Registos após limpeza: {after_rows:,}")
print(f"Percentagem removida: {removed_rows / before_rows * 100:.2f}%")

Limpeza concluída.
Registos antes: 3,475,226
Registos removidos: 153,999
Registos após limpeza: 3,321,227
Percentagem removida: 4.43%


## Questão 11 -  Volte a medir o consumo de memória após as otimizações e calcule a percentagem de redução obtida.

In [107]:
memory_after_mb = df_clean.memory_usage(deep=True).sum() / (1024 ** 2)
reduction_mb = memory_before_mb - memory_after_mb
reduction_pct = (reduction_mb / memory_before_mb * 100) if memory_before_mb > 0 else 0

print(f"Memória antes: {memory_before_mb:.2f} MB")
print(f"Memória após otimizações/limpeza: {memory_after_mb:.2f} MB")
print(f"Redução absoluta: {reduction_mb:.2f} MB")
print(f"Redução percentual: {reduction_pct:.2f}%")

Memória antes: 616.31 MB
Memória após otimizações/limpeza: 469.46 MB
Redução absoluta: 146.85 MB
Redução percentual: 23.83%
